## Import variables

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.base import clone
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Load dataset

Paper: https://uksim.info/uksim2015/data/8713a015.pdf

Dataset: https://archive.ics.uci.edu/dataset/363/facebook+comment+volume+dataset

The following dataset contains instances of facebook posts. The task is to predict how many comments the post will receive. 

We first load the whole training dataset with its columns. We join the training and testing set into one, as we will be manually dividing the dataset.

In [9]:
columns = ['Page Popularity/likes', 'Page Checkins', 'Page talking about', 'Page Category', 'Derived Feature 5', 'Derived Feature 6', 'Derived Feature 7', 
    'Derived Feature 8', 'Derived Feature 9', 'Derived Feature 10', 'Derived Feature 11', 'Derived Feature 12', 'Derived Feature 13', 
    'Derived Feature 14', 'Derived Feature 15', 'Derived Feature 16', 'Derived Feature 17', 'Derived Feature 18', 'Derived Feature 19', 'Derived Feature 20', 
    'Derived Feature 21', 'Derived Feature 22', 'Derived Feature 23', 'Derived Feature 24', 'Derived Feature 25', 'Derived Feature 26', 'Derived Feature 27', 
    'Derived Feature 28', 'Derived Feature 29', 'CC1', 'CC2', 'CC3', 'CC4', 'CC5', 'Base time', 'Post length', 'Post Share Count', 'Post Promotion Status',
    'H Local', 'Post Published Sunday', 'Post Published Monday', 'Post Published Tuesday',  'Post Published Wednesday', 'Post Published Thursday', 
    'Post Published Friday', 'Post Published Saturday', 'Base DateTime Sunday', 'Base DateTime Monday', 'Base DateTime Tuesday','Base DateTime Wednesday', 
    'Base DateTime Thursday', 'Base DateTime Friday', 'Base DateTime Saturday', 'Target Variable' ]

training1 = pd.read_csv('./Dataset/Training/Features_Variant_1.csv', sep=',', header=None, names=columns)
training2 = pd.read_csv('./Dataset/Testing/Features_TestSet.csv', sep=',', header=None, names=columns)

training_complete = pd.concat([training1, training2])
training_complete.shape

(50993, 54)

### Data Preprocessing
In this part we clean the dataset, remove columns, and adapt values for our model training.
We dropped the post promotion status because that column is filled with all zeros.
We also drop the duplicate rows, which are all the rows that are repeated. These rows and columns don't add any additional information as seen in the data analysis notebook.

Scaling and modifying outliers was considered in other models, but in this case we didn't do scaling because decision trees are not affected by this. Also, decision trees are robust against outliers so we don't address that either.

In [10]:
training = training_complete.drop("Post Promotion Status", axis='columns') # Drop column that has only zeros
training = training.drop_duplicates() # drop duplicates in our dataset
'''
The .values changes type from pandas to a numpy array, it strips the labels of rows and columns
'''
X = training.iloc[:, :-1]
y = training.iloc[:, -1]

X_np = training.iloc[:, :-1].values # Independent variables - the features.
y_np = training.iloc[:, -1].values # dependent variable - prediction

Separate the features into training and testing set

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size = 0.2, random_state = 1) # 20% for testing dataset

## Experiements
The next are options that were considered to train and test our model. This are the main experiements that we will be running with decision trees in which we have different hypothesis regarding the results of each different way that the model is trained. We will evaluate them, understand their result, and try to find a model with the most improved metrics.

#### Option 1
In here, we create a function that trains a decision tree. Based on that, it uses the function of entropy to rank the features that better divide the data. We iterate and train multiple models with the top X features to then compare results.
This function just gets an instance of a model of decision tree, we rank the best features based on the top features that reduce entropy the most, and then use the top X features (defined by the function) to train a model. In here we are trying naive feature selection to reduce the dimensionality of our data, because some features, as seen in data analysis, are not correlated, and we want to reduce so there is less noise and see if the models improved.

In [12]:
def topXFeaturesDT(model_instance, X_, y_, base_model=None, feature_counts=range(5,25, 3)):
    '''
    Evaluates a model with different numbers of top features. We iterate on the number of features selected. 
    The 'top features' are evaluated from a decision tree model that ranks the features that caused the most information gained. 

    Parameters:
    - X_: dataframe with all the values (training + testing)
    - y_: dataframe with all target values (training + test)
    - base_model: any decision tree model that is then used to rank the most important features
    - feature_counts: number of top X features/columns that we are trying
    '''
    results = []

    if base_model is None:
        base_model = DecisionTreeRegressor()
        base_model.fit(X_, y_)
        
    # Ranks most important features based on prev model
    feature_importance = pd.DataFrame({
    'Feature': X_.columns,
    'Importance': base_model.feature_importances_
    }).sort_values('Importance', ascending=False)

    X_train_full, X_test_full, y_train_top, y_test_top = train_test_split(X_, y_, test_size=0.2, random_state=42)

    # Select the x most important features
    for x in feature_counts:
        print(f"\n--- Model with top {x} features ---")
        top_X_features = feature_importance['Feature'].head(x).tolist()
        
        X_top_train = X_train_full[top_X_features]
        X_top_test = X_test_full[top_X_features]

        model = clone(model_instance)
        model.fit(X_top_train, y_train_top)
    
        test(model, X_top_test, y_test_top)
        results.append({
            'feature_count': x,
            'features': top_X_features,
            'model': model,
        })

    return results
    

#### Option 2
Similar to the option 1 before, but in here the number of features to train are fixed at the start.

In [68]:
def topXFeaturesDTStatic(model_instance, X_, y_, num_features, base_model=None, columns=None):
    '''
    Evaluates a model with top num_features passed.

    Parameters:
    - X_: dataframe with all the values (training + testing)
    - y_: dataframe with all target values (training + test)
    - base_model: any decision tree model that is then used to rank the most important features
    - num_features: the number of features that will be used to train the model
    - feature_counts: number of top X features/columns that we are trying
    '''

    X_train_full, X_test_full, y_train_top, y_test_top = train_test_split(X_, y_, test_size=0.2)

    if base_model is None:
        base_model = DecisionTreeRegressor()
        base_model.fit(X_train_full, y_train_top)
        
    # Ranks most important features based on prev model
    feature_importance = pd.DataFrame({
    'Feature': X_.columns,
    'Importance': base_model.feature_importances_
    }).sort_values('Importance', ascending=False)

    # Select the x most important features
    print(f"\n--- Model with top {num_features} features ---")
    top_X_features = feature_importance['Feature'].head(num_features).tolist()
    if columns is not None:
        top_X_features = columns
    
    X_top_train = X_train_full[top_X_features]
    X_top_test = X_test_full[top_X_features]

    model = clone(model_instance)
    model.fit(X_top_train, y_train_top)

    test(model, X_top_test, y_test_top)
    results = {
        'feature_count': num_features,
        'features': top_X_features,
        'model': model,
    }

    return results
    

### Option 3

When data is linearly separable, linear models like linear regression, logistic regression, or linear SVMs can more effectively separate the classes or predict values. We will try to improve the linear separability of our data to enhance model performance.

This approach is typically used for linear models like linear regression. However, we will also try it with decision trees to see how it performs. We're implementing this because many features in our dataset are uncorrelated with our target variable as shown in the data analysis notebooks. By applying PCA, we hope to see improved results.

In [14]:
def DT_PCA(model_instance, X_, y_):
    '''
    The function receives a model and features/target variables and trains the model after applying PCA to the features

    Parameters: 
    - model_instance: model to train
    - X_: feautres
    - y_: target variables
    '''
    # 1. Split data
    X_train_full, X_test_full, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42)
    
    # 2. Standardize features FIRST
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_full)
    X_test_scaled = scaler.transform(X_test_full)
    
    # 3. THEN apply PCA to scaled data
    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    
    # 4. Train model (PCA output doesn't need further scaling)
    model = clone(model_instance)
    model.fit(X_train_pca, y_train)
    test(model, X_test_pca, y_test)
    
    results = {
        'pca_components': X_train_pca.shape[1],
        'model': model,
    }
    return results

## Test - Creating functions
The next function have the function to be used to test the model with various metrics and compare the different models.

Explanation of metrics used:
- rmse (inf): average magnitude of prediction errors, but in the same units as your target variable. penalizes more big errors. i.e. penalizes more being
- 100 comments off than 50 comments from real value
- mae (inf): average absolute difference between predicted and real comment value
- r2 (0-1): compares model error against as if you predicted average values. 0.65 means 65% of the variance in comment counts. aiming >0.7
- mabse (inf): median abolsute error orders all errors and returns the median/middle value. This helps when values are skewed
- smape(0-200): percentage of: average of (absolute difference between y_pred and y_test) / (average value of y_pred,y_test)

These metrics give us a comprehensive view of model performance, with each metric highlighting different aspects of prediction accuracy.

In [16]:
def calculate_smape(y_true, y_pred):
    '''
    Calculate the smape which is the percentage of: average of (absolute difference between y_pred and y_test) / (average value of y_pred,y_test)

    Params:
    y_true: the real values of the dataset
    y_pred: the predictions of our model
    '''
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred))
    
    zero_indexes = denominator == 0
    
    valid_entries = ~zero_indexes
    if not np.any(valid_entries):
        return 0.0
    
    smape_values = 2 * numerator[valid_entries] / denominator[valid_entries] * 100
    return np.mean(smape_values)
    
def test(model, X_test_=X_test, y_test_=y_test, y_pred=None):
    '''
    We test our model and print various metrics for comparison

    Params:
    model: to test
    X_test: which are features to test
    y_test: the real values that match X_test
    '''
    if y_pred is None:
        y_pred = model.predict(X_test_)
    
    rmse = root_mean_squared_error(y_test_, y_pred)
    mae = mean_absolute_error(y_test_, y_pred)
    mse = mean_squared_error(y_test_, y_pred)
    mabse = median_absolute_error(y_test_, y_pred)
    r2 = r2_score(y_test_, y_pred)
    smape = calculate_smape(y_test_, y_pred)

    print(f"Root mean Squared Error: {rmse:.4f}")
    print(f"Mean absolute Error: {mae:.4f}")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"Median absolute Error: {mabse:.4f}")
    print(f"R² Score: {r2:.4f}")
    print(f"sMAPE: {smape:.4f}")
    

## Training
In this part, we train the model and for each model we evaluate the results.

### Baseline model
#### Model with all the features
This is the baseline model using all available features. After very simple data preprocessing, we compute the testing values to evaluate performance. This will be our baseline model which we aim to improve upon through iteration. We believe decision tree is a good choice because it is robust against outliers which is one of the main problems our dataset has.

In [26]:
dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(X_train, y_train)
test(dt_regressor)

Root mean Squared Error: 53.9661
Mean absolute Error: 9.2336
Mean Squared Error: 2912.35
Median absolute Error: 1.0000
R² Score: 0.0546
sMAPE: 116.3375


#### Decision tree Regressor with top X features
After establishing our baseline model, the next code iterates through the same model but with different feature subsets. The function selects the best features (those that provide the most information gain) and uses the top X features to evaluate the model. We do this to see if there is any improvement, as we expect the model to be more accurate with fewer features since many are uncorrelated.

In [125]:
dt_regressor_top = DecisionTreeRegressor()
dt_regressor_top = topXFeaturesDTRegressor(dt_regressor_top, X, y, dt_regressor)


--- Model with top 5 features ---
Root mean Squared Error: 58.5504
Mean absolute Error: 10.5416
Mean Squared Error: 3428.15
Median absolute Error: 1.0000
R² Score: 0.3011
sMAPE: 122.7723

--- Model with top 8 features ---
Root mean Squared Error: 61.6623
Mean absolute Error: 10.0507
Mean Squared Error: 3802.24
Median absolute Error: 1.0000
R² Score: 0.2249
sMAPE: 116.9483

--- Model with top 11 features ---
Root mean Squared Error: 64.0536
Mean absolute Error: 9.9791
Mean Squared Error: 4102.87
Median absolute Error: 1.0000
R² Score: 0.1636
sMAPE: 116.9760

--- Model with top 14 features ---
Root mean Squared Error: 58.5225
Mean absolute Error: 9.2960
Mean Squared Error: 3424.89
Median absolute Error: 1.0000
R² Score: 0.3018
sMAPE: 116.6560

--- Model with top 17 features ---
Root mean Squared Error: 62.5704
Mean absolute Error: 9.8812
Mean Squared Error: 3915.06
Median absolute Error: 1.0000
R² Score: 0.2019
sMAPE: 116.5879

--- Model with top 20 features ---
Root mean Squared Error:

#### Results
As seen in the metrics, the measure that improved most was the R² score when using the top 14 features, which means the model is capturing a better proportion of the variance in the target variable.
The best results achieved for the plain decision tree regressor is the model with the top 14 features, with an R² Score of 0.3018, which is a significant improvement from the previous model's score of 0.05. This could be due to the reduction of the number of features - many irrelevant features that caused noise are removed, improving the overall metrics of the model.

### Random Forest
As seen in class, we train the random forest (bagging) which builds many decision trees and then predicts based on the average of all individual predictions. This reduces variance in our model. We apply the same experiments as before, training with all the data and then with only the top features.

Random forests are valuable for this dataset because they combine the robustness of decision trees with improved generalization through ensemble learning. By creating multiple trees with different subsets of data and features, random forests can better handle the complex relationships in our Facebook comment prediction task while being less prone to overfitting than individual decision trees.

We train with all the data available. 

In [126]:
# Create and train the model
rf_model = RandomForestRegressor(
    n_estimators=100,  # Number of trees
    max_features='sqrt',  # Try sqrt(n_features) at each split
    random_state=42
)
rf_model.fit(X_train, y_train)
test(rf_model)

Root mean Squared Error: 40.2087
Mean absolute Error: 7.2797
Mean Squared Error: 1616.74
Median absolute Error: 0.7700
R² Score: 0.4752
sMAPE: 130.1362


We now train a random forest regressor but with the top X features. 

In [127]:
rf_model = RandomForestRegressor(
    n_estimators=100,  # Number of trees
    max_features='sqrt',  # Try sqrt(n_features) at each split
    random_state=42
)
dt_regressor_top = topXFeaturesDTRegressor(rf_model, X, y, dt_regressor)


--- Model with top 5 features ---
Root mean Squared Error: 48.4196
Mean absolute Error: 8.0247
Mean Squared Error: 2344.46
Median absolute Error: 0.7100
R² Score: 0.5221
sMAPE: 132.7389

--- Model with top 8 features ---
Root mean Squared Error: 46.1015
Mean absolute Error: 7.3203
Mean Squared Error: 2125.35
Median absolute Error: 0.7200
R² Score: 0.5667
sMAPE: 131.0601

--- Model with top 11 features ---
Root mean Squared Error: 46.9869
Mean absolute Error: 7.3963
Mean Squared Error: 2207.76
Median absolute Error: 0.7000
R² Score: 0.5499
sMAPE: 130.2246

--- Model with top 14 features ---
Root mean Squared Error: 47.0395
Mean absolute Error: 7.3805
Mean Squared Error: 2212.72
Median absolute Error: 0.7100
R² Score: 0.5489
sMAPE: 129.5320

--- Model with top 17 features ---
Root mean Squared Error: 46.9154
Mean absolute Error: 7.3760
Mean Squared Error: 2201.05
Median absolute Error: 0.7000
R² Score: 0.5513
sMAPE: 129.8292

--- Model with top 20 features ---
Root mean Squared Error: 4

#### Results
Overall, when using the Random Forest regressor, models with fewer features (especially with the top 5-17 features) have:

Higher RMSE and MSE (which normally indicates worse performance)
But higher R² scores (which indicates better performance)

We get higher MSE but lower mean absolute error overall. We believe this is because we are reducing the variance and we are doing worse at predicting outliers with big amount of comments, which are causing the MSE to increase. However, the median absolute error orders the errors and picks the one in the middle, which is better when we have high skewness, which in our case we do (many instances have very low comment numbers with few having high numbers). Thus, we can see that we are doing worse when the post has high amount of comments but better with the overall posts. 

### Gradient Boosting
XGBoost is an optimized implementation of boosting (algorithm seen in class), where it builds an ensemble method to reduce bias. We believe that we could improve our results more than with random forest by using boosting as our dataset, because it is very dispersed, could need a more complex model, thus we need to reduce bias. Even more so, XGBoost includes regularization parameters to prevent variance from increasing.

The max_depth (3) helps prevent overfitting on outlier comment counts, while the learning_rate (0.1) ensures steady improvement without missing optimal solutions. For our Facebook comment prediction task, these settings balance the need for a complex enough model to capture the relationship between post features and comment volume, while avoiding overfitting to the highly skewed distribution of comments. We will use these results for comparison against other hyperparameter tunning. 

We train the model with all the data

In [129]:
# Create and train the model
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)
xgb_model.fit(X_train, y_train)
test(xgb_model)

Root mean Squared Error: 37.9187
Mean absolute Error: 7.5025
Mean Squared Error: 1437.83
Median absolute Error: 0.9971
R² Score: 0.5332
sMAPE: 144.9285


Then we train the model with only the top X features

In [131]:
xgb_model_top = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)
xgb_model_top_results = topXFeaturesDT(xgb_model_top, X, y, dt_regressor)


--- Model with top 5 features ---
Root mean Squared Error: 49.8174
Mean absolute Error: 8.4805
Mean Squared Error: 2481.77
Median absolute Error: 0.9353
R² Score: 0.4941
sMAPE: 145.2685

--- Model with top 8 features ---
Root mean Squared Error: 45.7413
Mean absolute Error: 7.9815
Mean Squared Error: 2092.27
Median absolute Error: 0.7826
R² Score: 0.5735
sMAPE: 144.2159

--- Model with top 11 features ---
Root mean Squared Error: 46.3077
Mean absolute Error: 7.9661
Mean Squared Error: 2144.41
Median absolute Error: 0.9148
R² Score: 0.5628
sMAPE: 142.6087

--- Model with top 14 features ---
Root mean Squared Error: 46.6957
Mean absolute Error: 7.9914
Mean Squared Error: 2180.49
Median absolute Error: 0.8937
R² Score: 0.5555
sMAPE: 141.9464

--- Model with top 17 features ---
Root mean Squared Error: 46.5560
Mean absolute Error: 7.9538
Mean Squared Error: 2167.46
Median absolute Error: 0.9332
R² Score: 0.5581
sMAPE: 141.4882

--- Model with top 20 features ---
Root mean Squared Error: 4

#### Results
Similar to our findings with Random Forest, the XGBoost models with fewer features (particularly with 8 features) demonstrate interesting performance characteristics:

Higher RMSE and MSE compared to the full-feature model, but significantly improved R² scores
The model with top 8 features achieves the best R² score of 0.5735, outperforming both the full-feature XGBoost model (0.5332) and the best Random Forest model

We see that overall the XGBoost is able to get better R² scores and median errors with 8 features. This again is because boosting algorithms excel at reducing bias in predictions by sequentially focusing on difficult examples. The top 8 features provide sufficient signal while eliminating noise that might confuse the model.
The slightly higher sMAPE values across XGBoost models suggest that while it's better at capturing overall patterns (higher R2), it may sometimes make proportionally larger errors on posts with very few comments. However, the improved median absolute error indicates that for most posts, the prediction accuracy is better than with Random Forest.

## Final model
From previous iterations, we found that models with 7-15 features demonstrated the best overall metrics. Now, we'll train both XGBoost and RandomForest using this feature range while employing cross-validation to identify optimal hyperparameters.

In [55]:
num_features = 10 # we choose top 10 number of features to train the model

For RandomForest, we're exploring key hyperparameters including:

Number of trees (50-200)
Maximum tree depth (None-30)
Minimum samples required for node splitting and leaf creation
Feature selection strategies (sqrt, log2, percentage)
Bootstrap sampling options
Impurity thresholds

In [20]:
rf_model_grid = RandomForestRegressor(random_state=42)

# All possible hypterparameters to configure based on documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
param_grid = {
    'n_estimators': [50, 100, 200],                  # num trees
    'max_depth': [None, 10, 20, 30],                 # set the max depth
    'min_samples_split': [2, 5, 10],                
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', 0.33],    
    'bootstrap': [True, False],                      # whether to use bootstrapping or not
    'min_impurity_decrease': [0.0, 0.01] 
}

# Create the cross-vaidation method
grid_search = GridSearchCV(
    estimator=rf_model_grid,
    param_grid=param_grid,
    cv=5,                               # 5 fold cross validation
    scoring='neg_mean_squared_error',   
    n_jobs=-1,                          
    verbose=1
)

results = topXFeaturesDTStatic(grid_search,X, y, num_features, dt_regressor)
print("Best parameters:", results['model'].best_estimator_)


--- Model with top 10 features ---
Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
Root mean Squared Error: 46.7323
Mean absolute Error: 7.1752
Mean Squared Error: 2183.90
Median absolute Error: 0.6800
R² Score: 0.5548
sMAPE: 132.7409
Best parameters: RandomForestRegressor(bootstrap=False, max_depth=30, max_features='sqrt',
                      min_samples_leaf=2, min_samples_split=5, random_state=42)


For XGBoost, we're tuning:

Number of boosting rounds (50-200)
Learning rate (0.01-0.3)
Tree depth and complexity controls
Sampling parameters for observations and features
Regularization parameters (alpha and lambda) to prevent overfitting

In [51]:
xgb_model_grid = xgb.XGBRegressor(random_state=42)

# All possible hypterparameters to configure based on documentation: https://xgboost.readthedocs.io/en/stable/parameter.html
param_grid = {
    'n_estimators': [50, 100, 200],        
    'learning_rate': [0.01, 0.1, 0.3],      
    'max_depth': [3, 5, 7],                  # Depth of trees
    'min_child_weight': [1, 3, 5],           
    'gamma': [0, 0.1, 0.3],           
    'subsample': [0.7, 0.8, 1.0],           
    'colsample_bytree': [0.7, 0.8, 1.0],  
    # regularization to penalize large values. mentioned it we saw it in class
    'reg_alpha': [0, 0.1, 1],                
    'reg_lambda': [0.1, 1, 10]             
}

# Create GridSearchCV object
grid_search_xg = GridSearchCV(
    estimator=xgb_model_grid,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)
#dimensionality reduction
results = topXFeaturesDTStatic(grid_search_xg,X, y, num_features, dt_regressor)
print("Best parameters:", results['model'].best_estimator_)

# Saved the model
# from joblib import dump, load
# dump(results['model'], f'model_xgboost.joblib')

For repeating the previous metrics, you can run the following code to train the model and see the results. 

In [14]:
saved_cross_validation = load('./Models/model_xgboost.joblib')

model_columns = ['CC2','Base time','Post Share Count','Derived Feature 25','H Local','Post length','Derived Feature 14','Page Popularity/likes',
                'Derived Feature 22','Derived Feature 7']
# Get the best parameters from saved grid search
best_params = saved_cross_validation.best_estimator_.get_params()

# Create new model with these parameters
best_model = xgb.XGBRegressor(**best_params)
num_features = 10
results = topXFeaturesDTStatic(best_model, X, y, num_features, columns=model_columns)
model = results['model']

X_filtered = X[model_columns]
X_train_temp, X_test_temp, y_train_temp, y_test = train_test_split(X_filtered, y_np, test_size = 0.2) # 20% for testing dataset
test(model, X_test_temp, y_test)


--- Model with top 10 features ---
Root mean Squared Error: 25.8804
Mean absolute Error: 5.5586
Mean Squared Error: 669.79
Median absolute Error: 0.8810
R² Score: 0.8040
sMAPE: 137.4270


### Conclusion
After comprehensive experimentation with feature selection and hyperparameter tuning, our key findings include:

XGBoost significantly outperforms Random Forest: Our optimized XGBoost model achieved an R² score of 0.8040, explaining 80% of the variance in comment volumes - a substantial improvement over both baseline models and optimized Random Forest (R² of 0.5548).
Feature reduction improved performance: Using only the top 8-10 features consistently outperformed using the full feature set, confirming our hypothesis that many features were irrelevant or noisy.
Error metrics trade-offs: XGBoost showed better RMSE, MAE, and MSE, while Random Forest had slightly better median absolute error, suggesting XGBoost handles the full range of values better.
Bias-variance explanation: XGBoost's superior performance indicates our problem needed bias reduction more than variance reduction, aligning with the complex relationships in our dataset.

Based on these findings, we select the optimized XGBoost model with the top 10 features as our best model, demonstrating strong predictive capability for Facebook comment volume prediction.

## Trying to improve linear separability
For our final experiment, we applied PCA to our dataset before training with our best XGBoost model (identified through previous cross-validation). PCA transforms the data into uncorrelated components that might improve linear separability.

As seen in the results below, the PCA transformation significantly decreased performance compared to our optimized XGBoost model (R² dropped from 0.8040 to 0.3152). This suggests that the non-linear relationships captured by XGBoost on the original features are lost when transforming to principal components. While PCA creates orthogonal features that might benefit linear models, our best-performing model (XGBoost) already handles feature interactions effectively through tree-based structures. The results confirm that for this Facebook comment prediction task, feature selection is a more effective dimensionality reduction approach than PCA transformation.

In [72]:
saved_cross_validation = load('model_xgboost.joblib')

# Get the best parameters from saved grid search
best_params = saved_cross_validation.best_estimator_.get_params()

# Create new model with these parameters
best_model = xgb.XGBRegressor(**best_params)

results = DT_PCA(best_model, X, y)
results['pca_components']

Root mean Squared Error: 57.9580
Mean absolute Error: 10.9336
Mean Squared Error: 3359.12
Median absolute Error: 1.7670
R² Score: 0.3152
sMAPE: 147.5719


25

# Extra features
For this last experiment, we will use this option after finding the optimal model using the normal dataset. This option contains all the features created in the notebook featureEngineering.ipynb. It creates new features that provide more valuable information. All this data will be used to train the best decision tree obtained using all the other experiments to see if there is any improvement.

For better understanding of these features, more explanation is provided on the notebook.

In [74]:
train_df = pd.read_csv("train_df.csv")
test_df = pd.read_csv("test_df.csv")


X_train = train_df.drop(columns='Target_Comment_Volume')
y_train = train_df['Target_Comment_Volume']

X_test  = test_df.drop(columns='Target_Comment_Volume')
y_test  = test_df['Target_Comment_Volume']

In [75]:
saved_cross_validation = load('./Models/model_xgboost.joblib')

# Get the best parameters from saved grid search
best_params = saved_cross_validation.best_estimator_.get_params()

# use one-hot encoding
categorical_cols = ['Page_Category', 'Post_Promotion_Status', 'Published_Day', 'BaseDate_Day']
X_train = pd.get_dummies(X_train, columns=categorical_cols)
X_test = pd.get_dummies(X_test, columns=categorical_cols)
missing_cols = set(X_train.columns) - set(X_test.columns)
for col in missing_cols:
    X_test[col] = 0
X_test = X_test[X_train.columns]

# Train model with all the features
print("<---Train model with all the features--->")
best_model = xgb.XGBRegressor(**best_params)
best_model.fit(X_train, y_train)
test(best_model, X_test, y_test)

X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

best_model1 = xgb.XGBRegressor(**best_params)
num_features = 10
results = topXFeaturesDTStatic(best_model1, X, y, num_features)

<---Train model with all the features--->
Root mean Squared Error: 14.6582
Mean absolute Error: 3.3010
Mean Squared Error: 214.86
Median absolute Error: 0.5319
R² Score: 0.7879
sMAPE: 140.6487

--- Model with top 10 features ---
Root mean Squared Error: 18.6941
Mean absolute Error: 3.6789
Mean Squared Error: 349.47
Median absolute Error: 0.5449
R² Score: 0.6854
sMAPE: 141.5594


### Conclusion extra feature engineering
After evaluating the impact of feature engineering on our model, the results show improvements in most performance metrics. Using the new features derived from the original dataset in featureEngineering.ipynb, our best XGBoost model achieved an R2 score of 0.7879, which is slightly lower than our previous best of 0.8040. However, other metrics showed improvements: RMSE decreased from 25.88 to 14.66, MSE dropped from 669.79 to 214.86, and MAE improved from 5.56 to 3.30.

This suggests that our engineered features (hourly rates, category-level target means, and ratio aggregates) particularly enhanced the model's ability to make accurate predictions across the entire dataset, especially for posts with typical comment volumes or when posts deactivated comments.

# Final conclusion
In this project, we aimed to predict the number of comments a Facebook post will receive using various decision tree regression models. We conducted several experiments to understand the data and improve prediction accuracy:
1. Baseline Model: Decision Tree Regressor
2. Random Forest with all and top features. Best results with top 14 features
3. XGBoost with all and top features. Best results with top 8 features. 

## Hyperparameter Tuning with Cross-Validation
We conducted extensive hyperparameter tuning to optimize our most promising models:

### RandomForest with Cross-Validation
Used GridSearchCV with 5-fold cross-validation
Tested 6,480 hyperparameter combinations
Best results:
Root mean Squared Error: 46.7323
Mean absolute Error: 7.1752
Mean Squared Error: 2183.90
Median absolute Error: 0.6800
R² Score: 0.5548
sMAPE: 132.7409

### XGBoost with Cross-Validation

Similar extensive hyperparameter search
Selected the optimal hyperparameters for our dataset
Best results: 
Root mean Squared Error: 25.8804
Mean absolute Error: 5.5586
Mean Squared Error: 669.79
Median absolute Error: 0.8810
R² Score: 0.8040
sMAPE: 137.4270

## PCA Experiment for Linear Separability
As a final experiment, we applied PCA to our dataset before training with our best XGBoost model, where performance declined significantly in all metrics.

## Extra Feature Engineering
As a final experiment, we created domain-specific features including hourly comment rates, category-level target encoding, and post engagement indicators, which significantly improved prediction accuracy for typical posts while maintaining comparable R² scores.

## Final model and insights
Our best model is the XGBoost regressor with all the extra feature engieering and optimized hyperparameters through cross-validation. From all our experiments, we concluded that feature selection is crucial as eliminating noise is very important. We also found that with this model, bias reduction was more important than variance reduction as me saw that having higher compelx model got better results that reducing variance (when we compared boosting vs random forest).